# Guardar y restaurar un modelo

Guarde y restaure un modelo utilizando TensorFlow.
Este ejemplo utiliza la base de datos de dígitos manuscritos del MNIST.
(http://yann.lecun.com/exdb/mnist/).

In [1]:
from __future__ import print_function

# Importar datos MINST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parametros
learning_rate = 0.001
batch_size = 100
display_step = 1
model_path = "/tmp/model.ckpt"

# Network Parametros
n_hidden_1 = 256 # Número de características de la 1ª capa
n_hidden_2 = 256 # Número de características de la 2ª capa
n_input = 784 # Entrada de datos MNIST (forma de la imagen: 28*28)
n_classes = 10 # Clases totales del MNIST (0-9 dígitos)

# tf Entrada de gráficos
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Crear modelo
def multilayer_perceptron(x, weights, biases):
    # Capa oculta con activación RELU
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Capa oculta con activación RELU
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Capa de salida con activación lineal
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Almacenar el peso y el sesgo de las capas
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construir modelo
pred = multilayer_perceptron(x, weights, biases)

# Definir la pérdida y el optimizador
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Inicialización de las variables
init = tf.global_variables_initializer()

In [4]:
# 'Saver' op para guardar y restaurar todas las variables
saver = tf.train.Saver()

In [5]:
# Ejecutar la primera sesión
print("Starting 1st session...")
with tf.Session() as sess:
    # Inicializar variables
    sess.run(init)

    # Ciclo de entrenamiento
    for epoch in range(3):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Circuito sobre todos los lotes
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Optimización de la ejecución op (backprop) y cost op (para obtener el valor de la pérdida)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Calcular la pérdida media
            avg_cost += c / total_batch
        # Visualizar logs por paso de época
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print("First Optimization Finished!")

    # Test modelo
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calcular la precisión
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

    # Guardar los pesos de los modelos en el disco
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)

Starting 1st session...
Epoch: 0001 cost= 187.778896380
Epoch: 0002 cost= 42.367902536
Epoch: 0003 cost= 26.488964058
First Optimization Finished!
Accuracy: 0.9075
Model saved in file: /tmp/model.ckpt


In [6]:
# Ejecutar una nueva sesión
print("Starting 2nd session...")
with tf.Session() as sess:
    # Inicializar variables
    sess.run(init)

    # Restaurar los pesos de los modelos previamente guardados
    load_path = saver.restore(sess, model_path)
    print("Model restored from file: %s" % save_path)

    # Reanudar el entrenamiento
    for epoch in range(7):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Circuito sobre todos los lotes
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Optimización de la ejecución op (backprop) y cost op (para obtener el valor de la pérdida)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Calcular la pérdida media
            avg_cost += c / total_batch
        # Visualizar logs por paso de época
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Second Optimization Finished!")

    # Test modelo
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calcular la precisión
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval(
        {x: mnist.test.images, y: mnist.test.labels}))

Starting 2nd session...
Model restored from file: /tmp/model.ckpt
Epoch: 0001 cost= 18.292712951
Epoch: 0002 cost= 13.404136196
Epoch: 0003 cost= 9.855191723
Epoch: 0004 cost= 7.276933088
Epoch: 0005 cost= 5.564581285
Epoch: 0006 cost= 4.165259939
Epoch: 0007 cost= 3.139393926
Second Optimization Finished!
Accuracy: 0.9385
